In [ ]:
!pip install -q playwright
!playwright install chromium

import nest_asyncio
import asyncio
from playwright.async_api import async_playwright

# 讓 Colab 可以重複使用 async loop（這是關鍵！）
nest_asyncio.apply()

async def scrape_pigepm():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True, args=["--no-sandbox"])
        page = await browser.new_page()
        await page.goto("https://pigepm.moa.gov.tw/", wait_until="networkidle")

        items = await page.query_selector_all('.number-name-item')
        farm_count = user_count = None

        for item in items:
            label = (await item.inner_text()).strip()
            value = (await item.evaluate("el => el.previousElementSibling.textContent")).strip()
            if '牧場數量' in label:
                farm_count = int(value)
            elif '使用者數量' in label:
                user_count = int(value)

        await browser.close()
        return farm_count, user_count

# 正確方式（在 Colab 中）
farm, user = await scrape_pigepm()

print("🐷 牧場數量：", farm)
print("👥 使用者數量：", user)

import gspread
import google.auth
from google.colab import auth
from datetime import datetime

# 1. 登入 Google 帳號（Colab 會跳出認證流程）
auth.authenticate_user()

# 2. 取得憑證並授權給 gspread
creds, _ = google.auth.default()
gc = gspread.authorize(creds)

# 3. 指定 Google Sheet ID 與工作表名稱
SHEET_ID = "1BRfNr84btjJFPH9CXUiTMeojHGb16Y7vRf_D92kHKOU"
WORKSHEET_NAME = "數據記錄"

sh = gc.open_by_key(SHEET_ID)
worksheet = sh.worksheet(WORKSHEET_NAME)

# 4. 組裝要寫入的資料
timestamp = datetime.now().strftime("%Y/%m/%d %H:%M:%S")
row = [timestamp, farm, user, "Playwright-Colab"]

# 5. 檢查標題列（可選）
if worksheet.cell(1, 1).value != "日期時間":
    worksheet.insert_row(["日期時間", "牧場數量", "使用者數量", "數據來源"], 1)

# 6. 寫入資料
worksheet.append_row(row)
print("✅ 已成功寫入 Google Sheet！")

import requests
from datetime import datetime

payload = {
    "farmCount": farm,
    "userCount": user,
    "source": "Playwright-Colab"
}

GAS_URL = "https://script.google.com/macros/s/AKfycbylRiww5xOBR3ElecBOl1Qv5pYGApwVGxXvrbdgWYIid7bQWjdQ_S4Npk29ZBtRNhmL6A/exec"  # 換成你的 Web App URL
r = requests.post(GAS_URL, json=payload)

print("✅ 已送出資料至 Google Sheet，回應：", r.text)



🐷 牧場數量： 986
👥 使用者數量： 1063
✅ 已成功寫入 Google Sheet！
✅ 已送出資料至 Google Sheet，回應： <!DOCTYPE html><html lang="en"><head><meta name="description" content="Web word processing, presentations and spreadsheets"><meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=0"><link rel="shortcut icon" href="//docs.google.com/favicon.ico"><title>Page Not Found</title><meta name="referrer" content="origin"><link href="//fonts.googleapis.com/css?family=Product+Sans" rel="stylesheet" type="text/css" nonce="BFb-Cbj1E4wwcj-a30pz9g"><style nonce="BFb-Cbj1E4wwcj-a30pz9g">.goog-inline-block{position:relative;display:-moz-inline-box;display:inline-block}* html .goog-inline-block{display:inline}*:first-child+html .goog-inline-block{display:inline}#drive-logo{margin:18px 0;position:absolute;white-space:nowrap}.docs-drivelogo-img{background-image:url(//ssl.gstatic.com/images/branding/googlelogo/1x/googlelogo_color_116x41dp.png);-webkit-background-size:1